Cargar los datos del dataset de “Shakespeare”, generar el proceso necesario para generar un
RDD o DataFrame con los siguientes Atributos (Word, Count, Group), en donde Word será
cada palabra única en el dataset, Count la cantidad de ocurrencias, y Group una agrupación de
3 categorías según la cantidad de palabras (p.e. “Frecuente”, “Intermedio”, “Poco Frecuente”),
luego se debe escribir en forma de parquet en cualquier ubicación del dbfs, particionado por el
campo Group, finalmente se debe materializar esta tabla dentro de DataWarehouse que provee
el entorno de databricks como tabla externa.

In [1]:
import findspark
findspark.init()
  
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
import re

In [2]:
def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())    

In [3]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("WordCount").getOrCreate()

In [4]:
path = "C:/Users/Usuario/Documents/GitHub/Data-Science/Challenges/Oiga_spark/100-0.txt"
file =spark.sparkContext.textFile("file:///{}".format(path))

In [17]:
words = file.flatMap(normalizeWords)
wordCounts =  words.map(lambda x:(x,1)).reduceByKey(lambda x, y: x+y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey()

results =  wordCountsSorted.collect()

In [11]:
words = wordCountsSorted.map(lambda x: Row(ID =int(x.split()[1])))

In [34]:
dataset = spark.createDataFrame(results).toPandas()

In [40]:
dataset = dataset.rename(columns={"_1":"Count", "_2":"Word"}).copy()

In [44]:
(dataset.Count.value_counts() / dataset.shape[0]).head(4)

1    0.392320
2    0.136394
3    0.075637
4    0.054139
Name: Count, dtype: float64

In [45]:
(dataset.Count.value_counts()).head(4)

1    10493
2     3648
3     2023
4     1448
Name: Count, dtype: int64

Se crean tres grupos para las siguientes proporciones:
+ Frecuente: mayor o igual a 10493
+ Intermedio: mayor o igual a 3648
+ Poco Frecuente: menor a 3648

In [48]:
import numpy as np

grupos = np.where((dataset.Count >= 10493), "Frecuente", "Intermedio")

In [51]:
dataset["Group"]= np.where((dataset.Count < 3648), "Poco Frecuente", grupos)
del grupos

In [53]:
dataset.Group.value_counts()

Poco Frecuente    26704
Intermedio           31
Frecuente            11
Name: Group, dtype: int64